In [163]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [164]:
# load data
data = pd.read_csv('./born_data.csv')

# separate x and y
y_df = data.iloc[:, -1].copy()
y_df[y_df <> 5] = 0
y_df[y_df == 5] = 1
x_df = data.iloc[:, 0:-1].copy()

In [165]:
for col in x_df.columns:
    if '11 years' in col or '16 years' in col:
        x_df.drop(col, axis = 1, inplace = True)

In [166]:
# if more than 15 unique values, then treated as numerical
master_list = []

for col in x_df.columns:
    # get unique count
    unique_count = len(x_df[col].unique())
    
    # numerical variable
    if unique_count > 15:
        # get a copy for processing
        curr_df = pd.DataFrame(x_df[col])
        # handle NaN
        curr_df[curr_df.isnull()] = -1
        # standardize
        curr_df = pd.DataFrame(scale(curr_df.values))
        # reset index
        curr_df.index = x_df.index
        # change column label
        curr_df.columns = [col]
        # append to master list
        master_list.append(curr_df)
    # categorical variable
    else:
        # get a copy
        curr_df = x_df[col].copy()
        # get dummy
        # if contains NaN
        if curr_df.isnull().values.any():
            dummy = pd.get_dummies(curr_df, dummy_na = True)
        # if no NaN
        else:
            dummy = pd.get_dummies(curr_df, dummy_na = False)
        # reset columns
        dummy.columns = [str(col)+'='+str(value) for value in dummy.columns]
        # reset index
        dummy.index = x_df.index
        # append to master list
        master_list.append(dummy)
        
# concatenate master list
x_df_extended = pd.concat(master_list, axis = 1)

print 'data shape before dummification/standardization', x_df.shape
print 'data shape after dummification/standardization', x_df_extended.shape

data shape before dummification/standardization (18558, 64)
data shape after dummification/standardization (18558, 532)


In [167]:
while True:
    x_train, x_test, y_train, y_test = train_test_split(\
        x_df_extended.values, y_df.values, test_size = 0.25)
    if sum(y_train == 1) > 2 and sum(y_test == 1) > 0:
        break
    else:
        print 'try again'

In [168]:
weight_list = [{1:value} for value in 10**np.arange(2, 12, 2)]
parameters = {'class_weight':weight_list}
gscv = GridSearchCV(LogisticRegressionCV(), parameters, scoring = 'roc_auc')
gscv.fit(x_train, y_train)
best_lr_model = gscv.best_estimator_

In [169]:
# get coefficient
coef = best_lr_model.coef_.flatten()

most_pos_var_index = np.argsort(coef)[-21:][::-1]
most_neg_var_index = np.argsort(coef)[0:3]

print 'most positive variables'
for index in most_pos_var_index:
    print coef[index], x_df_extended.columns[index]

most positive variables
6.69871788836 1M Maternal illness while pregnantM:C1-2=2.0
4.10681223576 0  Duration of labour-1st stage:hours=6.0
3.36365911889 0 Interval between this birth and last=6.0
3.00421999038 0  Inhalational analgesia=3.0
2.97389243636 0 Mother's weight in stones,1958=4.0
2.91049333114 0  Total number of antenatal vsits=2.0
2.80226269623 0 Abnormality during pregnancy=9.0
2.51668443628 Region at NCDS1 (1965) - 7 years=3.0
2.47044933444 Region at PMS (1958) - Birth=6.0
2.333402798 0 Birthweight-gestational age for sex=5.0
2.28337302387 0 Smoking prior to pregnancy=7.0
2.17683925142 Region at NCDS1 (1965) - 7 years=6.0
2.09527440089 0  Week of mothers 1st antenatal visit=nan
2.09064205565 0  Total number of antenatal vsits=5.0
2.08931074875 Region at PMS (1958) - Birth=3.0
2.07513382664 0 Hrs of work in pregnancy & wk stopped=1.0
2.0648116934 0  SEG maternal GPa as mum left school (GRO 1951)=14.0
1.9649183145 0 Interval between marriage & 1st birth=3.0
1.958993143 0 Wee

In [170]:
best_lr_model = gscv.best_estimator_

y_predict = best_lr_model.predict(x_test)
cm = confusion_matrix(y_test, y_predict)
accuracy_all = best_lr_model.score(x_test, y_test)
accuracy_epilepsy = float(cm[1,1])/float(cm[1,0]+cm[1,1])
accuracy_healthy = float(cm[0,0])/float(cm[0,0]+cm[0,1])

print accuracy_all
print accuracy_healthy
print accuracy_epilepsy

0.999568965517
1.0
0.0


In [171]:
print cm

[[4638    0]
 [   2    0]]


In [172]:
print gscv.best_params_
print gscv.best_score_

{'class_weight': {1: 10000000000}}
0.891444272778


In [ ]:
y_df.

In [173]:
#a function to deliver comparation
def compare(name):
    #ratio and count among whole population
    print pd.to_numeric(data[name],errors='coerce').value_counts()/pd.to_numeric(data[name],errors='coerce').value_counts().sum()
    print data[name].value_counts()
    
    #ratio and count among epilepsy patients
    print epi[name].value_counts()/epi[name].value_counts().sum()
    print epi[name].value_counts()

In [174]:
compare("0  Mums min. haemoglobin while pregnant")

10.0    0.568194
1.0     0.330745
2.0     0.082627
3.0     0.018375
4.0     0.000059
Name: 0  Mums min. haemoglobin while pregnant, dtype: float64
10.0    9586
1.0     5580
2.0     1394
3.0      310
4.0        1
Name: 0  Mums min. haemoglobin while pregnant, dtype: int64


NameError: global name 'epi' is not defined